In [2]:
import PySimpleGUI as sg
import pyaudio
import pygame
from pygame import mixer
from pydub import AudioSegment
import numpy as np
import os
import sounddevice as sd
import librosa
import threading

RATE = 44100
PITCH_SHIFT = 2
STREAM_THREAD = None  # Variable to store the stream thread

sg.theme('Material1')


def win():
    layout = [[sg.Text("Choose a status"), sg.Button('Wav Pitch'),
               sg.Button('MIC Pitch')]]
    return sg.Window('GUI', layout, relative_location=(0, -100), finalize=True)


def win1():
    layout = [[sg.Text("Choose a file: "), sg.Input(), sg.FileBrowse(key="-SOUND_PATH-")],
              [sg.Text(size=(12, 1), key='-STATUS-')],
              [sg.Button('Play', pad=(10, 0), key='-PLAY-'), sg.Button('Stop', pad=(10, 0), key='-STOP-'),
               sg.Text('Pitch: '),
               sg.Slider(range=(-12, 12), orientation='h', size=(50, 20), enable_events=True, key='-PITCH-',
                         default_value=1.0, resolution=1.0)
               ]]
    return sg.Window('WAV FILE', layout, relative_location=(0, -100), finalize=True)


def win2():
    layout = [[sg.Text("MIC PITCH SHIFTER"),
               sg.Text('Pitch: '),
               sg.Slider(range=(-12, 12), orientation='h', size=(50, 20), enable_events=True, key='-PITCH-',
                         default_value=1.0, resolution=1.0),
               sg.Button('Stop MIC', pad=(10, 0), key='-STOP-MIC-')
               ]]
    return sg.Window('MIC PITCH', layout, finalize=True)


def change_pitch(audio, semitones):
    return audio._spawn(audio.raw_data, overrides={
        "frame_rate": int(audio.frame_rate * (2 ** (semitones / 12.0)))
    })


def pitch_shift_callback(indata, outdata, frames, time, status):
    global RATE, PITCH_SHIFT
    if status:
        print('Status:', status)
    audio_data = np.frombuffer(indata, dtype=np.float32)
    shifted_audio = librosa.effects.pitch_shift(audio_data, sr=RATE, n_steps=PITCH_SHIFT)
    if len(shifted_audio) < len(indata):
        shifted_audio = np.pad(shifted_audio, (0, len(indata) - len(shifted_audio)))
    else:
        shifted_audio = shifted_audio[:len(indata)]
    outdata[:] = shifted_audio[:, np.newaxis]


mixer.init()
pygame.init()
is_playing = False


def pitch():
    window_pitch = win1()
    temp_file_path = ""
    try:
        while True:
            event, values = window_pitch.read()
            if event == sg.WIN_CLOSED or event == "Exit":
                mixer.quit()
                temp_file_path = values['-SOUND_PATH-'] + 'temp.wav'
                break
            sound_path = values["-SOUND_PATH-"]
            if not sound_path:
                sg.Popup("No song specified.")
                continue
            pitch_value = values['-PITCH-']
            audio = AudioSegment.from_file(sound_path)
            modified_audio = change_pitch(audio, pitch_value)
            if event == '-PLAY-':
                window_pitch['-STATUS-'].update('Playing')
                is_playing = True
                mixer.music.stop()
                pygame.mixer.quit()
                temp_file_path = sound_path + 'temp.wav'
                modified_audio.export(temp_file_path, format="wav")
                mixer.init()
                mixer.music.load(temp_file_path)
                mixer.music.play()
            elif event == '-STOP-':
                mixer.music.stop()
                is_playing = False
    finally:
        window_pitch.close()
        if os.path.exists(temp_file_path):
            os.remove(temp_file_path)


def set_pitch_shift(param):
    global PITCH_SHIFT
    PITCH_SHIFT = param


def stop_mic():
    global STREAM_THREAD
    if STREAM_THREAD:
        STREAM_THREAD.close()
    window_mic.close()


def mic():
    global STREAM_THREAD
    window_mic = win2()
    CHUNK = 1024  # Audio chunk size
    CHANNELS = 1

    def mic_callback(indata, outdata, frames, time, status):
        global RATE, PITCH_SHIFT
        if status:
            print('Status:', status)
        audio_data = np.frombuffer(indata, dtype=np.float32)
        shifted_audio = librosa.effects.pitch_shift(audio_data, sr=RATE, n_steps=PITCH_SHIFT)

        # If the shifted audio length is less than the chunk size, pad it
        if len(shifted_audio) < len(indata):
            shifted_audio = np.pad(shifted_audio, (0, len(indata) - len(shifted_audio)))
        else:
            shifted_audio = shifted_audio[:len(indata)]

        # Copy shifted audio data to output buffer
        outdata[:] = shifted_audio[:, np.newaxis]

    def stream_thread():
        with sd.Stream(channels=CHANNELS, callback=mic_callback, samplerate=RATE, blocksize=CHUNK):
            print('Pitch shifting started. Press Ctrl+C to stop.')
            input("Press Enter to stop...")

    # Start the stream in a separate thread
    STREAM_THREAD = threading.Thread(target=stream_thread)
    STREAM_THREAD.start()

    while True:
        event, values = window_mic.read()

        if event == sg.WIN_CLOSED or event == "Exit":
            break

        if event == "-PITCH-":
            set_pitch_shift(values["-PITCH-"])

        if event == "-STOP-MIC-":
            stop_mic()

    window_mic.close()


window_main = win()
while True:
    event, values = window_main.read()
    if event in (None, 'Quit'):
        break
    if event == 'Wav Pitch':
        try:
            pitch()
        except:
            sg.popup('Instance Ended, Continuing to wait')
    elif event == 'MIC Pitch':
        try:
            mic()
        except:
            sg.popup('Instance Ended, Continuing to wait')
window_main.close()


Pitch shifting started. Press Ctrl+C to stop.


C:\Users\2050p\anaconda3\Lib\site-packages\librosa\core\spectrum.py:257: UserWarning: n_fft=2048 is too large for input signal of length=1024
  warnings.warn(
C:\Users\2050p\anaconda3\Lib\site-packages\librosa\core\spectrum.py:257: UserWarning: n_fft=2048 is too large for input signal of length=1024
  warnings.warn(
C:\Users\2050p\anaconda3\Lib\site-packages\librosa\core\spectrum.py:257: UserWarning: n_fft=2048 is too large for input signal of length=1024
  warnings.warn(
C:\Users\2050p\anaconda3\Lib\site-packages\librosa\core\spectrum.py:257: UserWarning: n_fft=2048 is too large for input signal of length=1024
  warnings.warn(
C:\Users\2050p\anaconda3\Lib\site-packages\librosa\core\spectrum.py:257: UserWarning: n_fft=2048 is too large for input signal of length=1024
  warnings.warn(
C:\Users\2050p\anaconda3\Lib\site-packages\librosa\core\spectrum.py:257: UserWarning: n_fft=2048 is too large for input signal of length=1024
  warnings.warn(
C:\Users\2050p\anaconda3\Lib\site-packages\lib

Press Enter to stop...
